# Background
POSIX provides clocks, timers, and real-time signals to solve time mangement problems.
## 1.1. Clocks
**"Clocks"** are abstract modeling entities (implemented in the kernel) that provide current time. A program can query the time, and can set the time if it has elevated privileges. There are **two** types of clocks:
* **CLOCK_REALTIME**: The physical time of the world according to the computer
* **CLOCK_MONOTONIC**: The virtual time of the compute, set to 0 when the computer was started, and monotonically increasing since then. 

Note that the value of the **CLOCK_REALTIME** may not increase monotonically, it may be adjusted during the operation due to the fact that the computer's clock circuitry is not accurate. Internet may perferom such adjustments.

## 1.2. Timers
**"Timers"** are abstract modelling entities (implemented in the kernel) that can generate "events" that should be handled by the program. POSIX defines **three** built-in "interval timers" that are available in a process:
* **ITIMER_REAL**: this timer fires after a specific amount of **real** time, and then *optionally* fires repeatedly with a given periodicity. This is based on **CLOCK_REALTIME**
* **ITIMER_VIRTUAL**: this timer fires after a process consumes a specified amount of time, and then *optionally* fires repeatedly with a given periodcity. This is based on a clock that is running **only when the process is active**. The timer is **not** running on an operating system call.
* **ITIMER_PROF**: same as **ITIMER_VIRTUAL**, except the timer is running **also** when the process is doing the system call. 

When the timer "fires", it initiates a SW interrupt, called **"signal"** in POSIX and sends it to the process.

## 1.3. Signals
Signals are simililar to interrupts, but they can be handled in the process not the OS.

When a signal is generated inside of the kernel, the kernel will interrupt the execution of the process, and can invoke a "signal handler" function that was registered by the process for such occasions. The signal handler is like an interrupt handler except it is running in the context of the process, i.e. it has access to global variables, etc. 

In multi-threaded programs, signals can be delivered to the process or a specific thread. Per-thread signal delivery is rather complicated to setup, we won't worry about that. 

The signal handler we are using will be: wake up a thread that is waiting for this signal handler to run. In this lab, we will only use **```SIGALRM```**, which gets generated when an interval timer fires. 

To **register the signal handler**, you need to use the function:
```c
    void signal(int sig, void (*func)(int));
```
In this case, 
* ```SIGALARM``` is in ```sig```. 
* The function pointer should point to the signal handler that will be invoked upon the arrival to the signal. The argument to the function is the number code of signal triggering the chain of events.

## 1.4. Semaphores
When the timer fires the signal handler is called. The operating system performs this by
**interrupting** the currently **executing thread** and **invoking the signal handler**, similarly to what
happens in the case of hardware interrupts. 

This signal handler invocation is completely **asynchronous**, and can happen between any two instructions of the code. The signal handler has to somehow inform a specific thread that the event has arrived. 

The only legal way to do this in ```pthreads``` is to use **semaphores**: specifically: 
* a signal handler must **‘post’** an event on a **semaphore** that a **thread could be waiting on**. 
    * Note that mutexes or condition variables cannot be used for this purpose. 
    
Semaphores are available in pthreads using the ```sem_t``` data type and various operations as listed below (```include <semaphore.h>```)
```c
    int sem_init(sem_t *sem, int pshared, unsigned value);
    int sem_wait(sem_t *sem);
    int sem_post(sem_t *sem);
```

Semaphores are objects in the OS that can be shared across processes. They operate according to the classical P and V operations. They also have an associated integer counter.
### 1.4.1. ```sem_init()```
```sem_init()``` initializes a semaphore variable and its counter with a ```value```. The ```pshared``` variable is set to 0 in this lab (make the semaphore non-shared across processes). 
### 1.4.2. ```sem_wait()```
```sem_wait()``` locks the semaphore referenced by ```sem``` performing a semaphore lock **(P)** operation. 
* If the semaphore value is currently zero, then the calling thread will not return from the call to ```sem_wait()``` until it either locks the semaphore or the call is interrupted by a signal. 
    * Upon a successful return, the state of the semaphore is locked and will remain locked until the ```sem_post()```function is excecuted and returns successfully. 
    
### 1.4.3. ```sem_post()``` 
```sem_post``` unlocks the semaphore referenced by ```sem``` by performing a semaphore unlock **(V)** operation
* If the semaphore value resulting from this operation is positive, then no threads were blocked waiting for the semaphore to become unlocked. 
* If the value of the semaphore resulting from this operation is zero, then one of the threads blocked waiting for the semaphore will be allowed to return successfully from its call to ```sem_wait()```.

# 2. Usage
## 2.1. ```timeval``` data type
The relevant data structure in this lab are as follows (defined in ```<sys/time.h>```). The data type below defines a simple time value:
```c
/* A time value that is accurate to the nearest microsecond but also has a range of years. */
struct timeval {
    __time_t tv_sec; /* Seconds. */
    __suseconds_t tv_usec; /* Microseconds. */
 };
```

This data type is used for the arguments of timer operations such as:
```c
struct itimerval {
    /* Value to put into `it_value' when the timer expires. */
    struct timeval it_interval;
    /* Time to the next timer expiration. */
    struct timeval it_value;
}
```
In the above data type:
* ```it_interval``` is for setting the periodicity of the timer firings
* ```it_value``` is the initial delay for the firing

## 2.2. ```Timer```
Before setting up a timer the signal handler needs to be set up. The reason for this is that the timer may fire as soon as it is set up. If the generated signal is not handled, it may terminate the process. 

A timer can be set up as follows: 
```c
int setitimer(int which, const struct itimerval * value, struct itimerval *ovalue);
```

In this lab, 
* the first argument will be ```ITIMER_REAL```
* the second argument will configure the timer: it will fire after ```value->it_value```, then periodically after every ```value->it_interval```
* the third argument will receive the previous setting of the timer that was in effect prior to the call. Once the timer is setup, it is active, i.e. it can fire immediately. 

### 2.2.a. Summary of semaphores for lab5
As discussed in **1.4**, we will use semaphores for signal handler:

* A thread that is waiting for a timer firing should block on a semaphore via a ```sem_wait()``` operation. 
* The ```signal handler``` should unlock the thread using a ```sem_post()```
* To repeat, the signal handler is called asynchronously but it is running in a process, so it is very risky to manipulate any shared data or use synchoronization objects.  Unless the semaphore to issue excecute a ```sem_post()``` operation

## 2.3. Periodic Task pseudo code
Using the above tecnhiques, a task that needs to run periodically can be implemented by using the following template:

```c
    void *PeriodicTask(void *arg) {
        <initialize semaphore>;
        <starting periodic timer, period = T>;
        while (cond) {
            <read sensors>;
            <update outputs>;
            <update state variables>;
            <wait for next activation using sem_wait()>;
        }
    }
```

The singal handler is like:
```c
    static void timersignalhandler(int sig){
        /* do a sem_post() on the semaphore */
    }
```

# 3. More accurate clock w/ nano sleep
The above method is possibly not accurate enough for certain application. The
reason is that there is an inherent delay between the signal delivery and the unblocking of the
thread due to the fact that the timers tick with a finite resolution. Timers will never expire before
the requested time, instead expiring some short, constant time afterwards, depending on the
system timer resolution.

The **"inaccurate wakeup"** can be solved by waiting for the absolute arrival time of the next job. The 
**```clock_nanosleep()```** must be used:

```c
int clock_nanosleep(clockid_t c_id, int flags, const struct timespec *rqtp, struct timespec *rmtp);
```

* The first argument is the clock type to be used: we are using **CLOCK_REALTIME**
* The second argument is **TIMER_ABSTIME** (indicating absolute time)
* The third argument will hold the absolute value when the thread needs to be woken up.
* The fourth argument is always NULL in this case.

The data type we are using for nano second resolution is:
```c
struct timespec{
    __time_t tv_sec; /* Seconds. */
    __syscall_slong_t tv_nsec; /* Nanoseconds. */
};
```

To obtain the clock current time from the clock, we can use:
```c
    int clock_gettime(clockid_t clock_id, struct timespec *tp);
```
Where clock id should be ```CLOCK_REALTIME```

The code snippet below shows how to use nanosleep:
```c
static struct timespec r;
static int period;

static void wait_next_activation(void) {
    clock_nano_sleep(CLOCK_REALTIME, TIMER_ABSTIME, &r, NULL);
    timespec_add_us(&r, period);
}

int start_periodic_timer(uint64_t offs, int t) {
    clock_gettime(CLOCK_REALTIME, &r);
    timespec_add_us(&r, offs);
    period = t;
    
    return 0;
}
```
The helper:
```c
    void timespec_add_us(struct timespec *t, uint64_t delta) 
```
increases the timevalue ```t``` by ```delta``` microseconds.

# Tasks
* Download a simple condition variable code example: ```ex_ttest.c```. Create an Eclipse project for it, and compile it. Study the code and write down what you expect to see when you run it in the lab. As you see, the goal is that the ```timer_func()``` is executed every 200 msec.
* Call this example: ex_nanosleep.c Use the code from the ```ex_ttest.c``` example but rewrite it so it uses the nanosleep-based approach - no signals, no interval timers, and no semaphores. The thread simply has to run every 200 msec. Add logging messages to show the time when the ```timer_func()``` is called.
* Modify the code of ```ex_ttest.c``` and ```ex_nanosleep.c``` to measure and compare the accuracy of the nanosleep and signal-based approach for time-triggered execution. You will need to put in extra code the reads the clock when the thread is release again and computes how accurately it was scheduled. 

# Appendix: Example code

In [1]:
#include <pthread.h>
#include <semaphore.h>  /* semaphore operations */
#include <signal.h>
#include <unistd.h>
#include <stdio.h>
#include <errno.h>
#include <stdlib.h>
#include <stdint.h>
#include <sys/time.h>
#include <errno.h>

In [2]:
static sem_t timer_sem;

static unsigned int timer_stopped;
static pthread_t timer_thread;

static sem_t demo_over;

In [3]:
static void timersignalhandler(int sig)
{
    sem_post(&timer_sem);
}

In [4]:
void shutdown_timer()
{
    timer_stopped = 1;
    sem_post(&timer_sem); 
    pthread_join(timer_thread, 0); 
}

In [5]:
static void timer_func()
{
    static int i = 0;

    printf ("Timer called %d!\n", i);
    if (++i == 100) {
        shutdown_timer();
        sem_post(&demo_over)
    }
}

In [6]:
static void * timerthread(void *_)
{
    while (!timer_stopped) {
        int rc = sem_wait(&timer_sem);
        if (rc == -1 && errno == EINTR) 
            continue;
        if (rc == -1) {                  
            printf("timerthread failed on sem_wait\n");
            exit(-1);
        }
        timer_func();
    }
    return 0;
}

In [7]:
void init_timer(void)
{
    sem_init(&timer_sem, 0, 0); 
    pthread_create(&timer_thread, (pthread_attr_t*)0, timerthread, (void*)0); 
    signal(SIGALRM, timersignalhandler); 
}

In [8]:
void set_periodic_timer(long delay)
{
    struct itimerval tval = {.it_interval = { .tv_sec = 0, .tv_usec = delay },
                         .it_value = { .tv_sec = 0, .tv_usec = delay }};
    setitimer(ITIMER_REAL, &tval, (struct itimerval*)0);
}

In [9]:
sem_init(&demo_over, 0, 0);

init_timer();
set_periodic_timer(200000);

while (sem_wait(&demo_over) == -1 && errno == EINTR)
    continue;
return 0;

Timer called 0!
Timer called 1!
Timer called 2!
Timer called 3!
Timer called 4!
Timer called 5!
Timer called 6!
Timer called 7!
Timer called 8!
Timer called 9!
Timer called 10!
Timer called 11!
Timer called 12!
Timer called 13!
Timer called 14!
Timer called 15!
Timer called 16!
Timer called 17!
Timer called 18!
Timer called 19!
Timer called 20!
Timer called 21!
Timer called 22!
Timer called 23!
Timer called 24!
Timer called 25!
Timer called 26!
Timer called 27!
Timer called 28!
Timer called 29!
Timer called 30!
Timer called 31!
Timer called 32!
Timer called 33!
Timer called 34!
Timer called 35!
Timer called 36!
Timer called 37!
Timer called 38!
Timer called 39!
Timer called 40!
Timer called 41!
Timer called 42!
Timer called 43!
Timer called 44!
Timer called 45!
Timer called 46!
Timer called 47!
Timer called 48!
Timer called 49!
Timer called 50!
Timer called 51!
Timer called 52!
Timer called 53!
Timer called 54!
Timer called 55!
Timer called 56!
Timer called 57!
Timer called 58!
Timer c